In [1]:
import pandas as pd
pd.set_option('display.max_row',500)
pd.set_option('display.max_columns',100)
pd.set_option('display.unicode.east_asian_width',True)
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
def getRecommendation(cosine_sim):
    simScores = list(enumerate(cosine_sim[-1]))
    simScores = sorted(simScores, key=lambda x: x[1], reverse=True)
    simScores = simScores[1:11] # 0번은 무조건 같은 영화기 때문에 1번부터 
    hotelidx = [i[0] for i in simScores]
    RecHotellist = df_review_one_sentence.iloc[hotelidx]
    return RecHotellist

In [13]:
df_review_one_sentence = pd.read_csv('./cleaned_review/onesentence_hotel_review_final.csv', index_col=0) # onesentence_review -> 영화의 리뷰를 다 모아 한문장으로 이어붙임
print(df_review_one_sentence.head(10))
print(df_review_one_sentence.info())

                                 hotel_name  \
0                             호텔 힐튼부산   
1                      호텔 파크하얏트 부산   
2                       호텔 롯데 호텔 부산   
3                     호텔 코모도 호텔 부산   
4             호텔 베스트웨스턴 해운대 호텔   
5                           호텔 라발스호텔   
6                     호텔 해운대 센텀 호텔   
7  호텔 라마다 앙코르 바이 윈덤 부산 해운대   
8               호텔 코오롱 씨클라우드 호텔   
9                  호텔 호텔 더 마크 해운대   

                                                                     review_one_sentence  
0  한적하다 분위기 야외 서비스 훌륭하다 식당 부족하다 저렴하다 식사 필요 친절하다 깨...   
1  완벽하다 전망 굳이 저녁 식사 싶다 친절하다 시설 현대 아름답다 이용 기회 부산 파...     
2  무료 업그레이드 해주다 감사하다 야외 수영 생각 넓다 사우나 시설 야외 수영장 수질...    
3  등급 올리다 주다 더욱 편안하다 안락하다 감사하다 욕실 샤워 커튼 냄새 나다 확인 ...     
4  바다 편의 시설 시장 가깝다 침대 오션 기대하다 작다 테라스 정말 백프로 만족 보고...     
5  전망 아주 루프 배경 죽여주다 딱하다 아침 조식 생각 종류 많다 욕실 불편하다 루프...     
6  소파 식탁 파티션 공간 나누다 어서 넓다 보이다 지내다 더욱 편하다 확실하다 넓다 ...     
7  가성 편리하다 청결하다 식당 운영 좀더 개선 되어다 깔끔하다 시설 주변 먹거리 볼거...    
8  청결하다 해변 가깝다 서다 편리하다 아침 식이 비싸

In [4]:
print(df_review_one_sentence.review_one_sentence[0])
print(len(df_review_one_sentence.review_one_sentence[0]))

한적하다 분위기 야외 서비스 훌륭하다 식당 부족하다 저렴하다 식사 필요 친절하다 깨끗하다 바다 보이다 욕조 딱하다 발코니 샤워실 낙후 조식 먹다 사람 일회용 장갑 사람 끼다 정확하다 전달 아침식사 생각 다양하다 메뉴 구성 되어다 후기 비다 만족스럽다 크다 기대 방문 그렇다 방문 재다 용하다 가격 대비 만족 아침식사 생각 다양하다 메뉴 구성 되어다 후기 비다 만족스럽다 크다 기대 방문 그렇다 방문 재다 용하다 가격 대비 만족 아침 일찍 바다 커피한잔 말다 날씨 영향 가다 덥다 친절하다 하수 냄새 그거 빼다 만족 산책로 자리 맛있다 식사 음식 따뜻하다 라이브 요리 해주다 오후 사용 수영장 생각 물이 차갑다 아침 사용 인피니티 따뜻하다 넓다 사람 많다 전혀 사용 문제 사우나 이용 코로나 때문 사우나 시설 이용 불가능하다 따뜻하다 피로 풀다 생각 시설 별로 내부 상점 가게 비다 대부분 조용하다 숙박 이외 다양하다 볼거리 아쉬움 수영장 넓다 욕조 넓다 서비스 정신 투철 항상 친절하다 어메니티 종류 비다 세면 대쪽 냄새 나다 물이 내려가다 터널 저니 바다 어우러지다 리조트 산책로 조성 되어다 곳곳 고객 위해 신경 쓰다 묻다 역시 친절하다 입구 드라이브 쓰루 형식 진행 체온 체크 마다 바다 망하다 쉬다 자리 마련 놓다 신의 한수 유리창 너머 바다 맘껏 느끼다 여유 가지다 지하 터널 저니 서점 여느 서점 못지않다 규모 크다 맘껏 읽다 공간 워터 하우스 수기 변경 필요하다 보이다 씻다 나오다 수건 수기 바닥 물이 떨어지다 비닐봉지 금상첨화 겠다 시설 가격 내부 공간 넓다 고급스럽다 부산 여유 분위기 동네 바닷가 보이다 해외 놀다 기분 가격 비싸다 외곽 서울 접근 소요 조식 메뉴 많다 외곽 여유 힐링 딱이다 아난 비싸다 바다 풍경 오믈렛 음식 전반 짜다 조식 메뉴 부실하다 추후 이용 조식 이용 예정 지불 비용 포만 감다 소고기 맛있다 늦다 조식 음식 방문 오션 마운틴뷰 땅뷰 주변 편리 인피니티 해운대 한적하다 쉬다 최적 식당 다양 음해 깨끗하다 실내 식당 생각 별로 들어서다 그

In [5]:
print(df_review_one_sentence['hotel_name'][0])

호텔 힐튼부산


In [6]:
print(df_review_one_sentence[df_review_one_sentence['hotel_name']=='호텔 신라스테이 제주'])

               hotel_name  \
173  호텔 신라스테이 제주   
441  호텔 신라스테이 제주   
461  호텔 신라스테이 제주   

                                                                   review_one_sentence  
173  조식 맛있다 욕조 침대 편안하다 청소 상태 생각 냉장고 성능 별로 먹다 조식 나다 ...  
441  위치 조식 맛있다 욕조 침대 편안하다 청소 상태 생각 냉장고 성능 별로 먹다 조식 ...  
461  위치 조식 맛있다 욕조 침대 편안하다 청소 상태 생각 냉장고 성능 별로 먹다 조식 ...  


In [7]:
print(df_review_one_sentence[df_review_one_sentence['hotel_name']=='호텔 신라스테이 제주'].index[0])

173


In [8]:
hotel_idx = df_review_one_sentence[df_review_one_sentence['hotel_name']=='호텔 호텔 알레그리아 서귀포'].index[0]

In [9]:
Tfidf = TfidfVectorizer(sublinear_tf=True)
Tfidf_matrix = Tfidf.fit_transform(df_review_one_sentence['review_one_sentence'])
print(Tfidf_matrix.shape)
print(Tfidf_matrix)

(1017, 17054)
  (0, 15379)	0.032817441004358815
  (0, 216)	0.0256880664648933
  (0, 2294)	0.01620760658473279
  (0, 14108)	0.023814718956807802
  (0, 16094)	0.01866218728397513
  (0, 6248)	0.030535783907347066
  (0, 2783)	0.05265445611394925
  (0, 15792)	0.0256880664648933
  (0, 11382)	0.016020262294702502
  (0, 16313)	0.023159553485455227
  (0, 15211)	0.05691293863364877
  (0, 14268)	0.019524837266042743
  (0, 862)	0.03266447395917814
  (0, 9309)	0.03297326741161949
  (0, 9380)	0.03379933575498242
  (0, 15486)	0.04775439017883709
  (0, 3883)	0.05265445611394925
  (0, 11864)	0.04245248886496905
  (0, 10937)	0.029751914383592863
  (0, 16150)	0.019187033516972672
  (0, 769)	0.04245248886496905
  (0, 2851)	0.01563806525261972
  (0, 7765)	0.028020935035315823
  (0, 878)	0.02165580691046733
  (0, 5581)	0.024816298415662696
  :	:
  (1015, 10433)	0.4065937117141387
  (1015, 8886)	0.44005508298427287
  (1016, 8249)	0.552074982576242
  (1016, 13180)	0.36564312797997733
  (1016, 1092)	0.28068433

In [10]:
cosine_sim = linear_kernel(Tfidf_matrix[hotel_idx], Tfidf_matrix)
print(cosine_sim.shape)
print(cosine_sim)

(1, 1017)
[[0.15279742 0.17624468 0.16990366 ... 0.14771733 0.02896393 0.03791255]]


In [11]:
simScores = list(enumerate(cosine_sim[-1])) #-> 나중에 영화를 섞었을 때 어느 자리에 있었는지 알기 위해서 인덱스를 붙여줌
print(simScores)

[(0, 0.15279741892502902), (1, 0.17624468352055675), (2, 0.16990365743013483), (3, 0.19308351923277142), (4, 0.2097218965168608), (5, 0.20815407758234256), (6, 0.1714235305651107), (7, 0.20231651085881858), (8, 0.19594639924947063), (9, 0.1976434503308742), (10, 0.19749256897985934), (11, 0.17800238876916227), (12, 0.18804342124184747), (13, 0.17619801791534492), (14, 0.07466507037270266), (15, 0.20897698414554275), (16, 0.16534567773322814), (17, 0.15300840947708372), (18, 0.18655922780528972), (19, 0.20184357058712793), (20, 0.19587404635652195), (21, 0.17613578031103447), (22, 0.19116085866164742), (23, 0.19952940273444258), (24, 0.19779595173562345), (25, 0.17360283254494227), (26, 0.0834048657197582), (27, 0.163052952139793), (28, 0.09048317213428406), (29, 0.1863169094011466), (30, 0.1731734028377459), (31, 0.18084888886226919), (32, 0.18648570555043062), (33, 0.15145392672720448), (34, 0.09824513124485708), (35, 0.18452775452147402), (36, 0.2056349244205728), (37, 0.099007704444

In [12]:
print(getRecommendation(cosine_sim))

                      hotel_name  \
438  호텔 호텔 알레그리아 서귀포   
458  호텔 호텔 알레그리아 서귀포   
218   호텔 디 아일랜드 블루 호텔   
228           호텔 호텔 윈스토리   
859   호텔 디 아일랜드 블루 호텔   
857           호텔 호텔 윈스토리   
188       호텔 헤이, 서귀포 호텔   
456       호텔 헤이, 서귀포 호텔   
171           호텔 퍼스트70 호텔   
439           호텔 퍼스트70 호텔   

                                                                      review_one_sentence  
438  위치 적당하다 가성 진짜 프론트 직원 피드백 늦다 크다 불편 위치 수영장 코로나 때...    
458  위치 적당하다 가성 진짜 프론트 직원 피드백 늦다 크다 불편 위치 수영장 코로나 때...    
218  가성 시설 크다 히터 나오다 카운터 남자 대처 해주다 친절하다 펜트하우스 업글 해주...   
228  원래 청결 예민 한편 여기다 깨끗하다 가격 저렴하다 방도 이쁘다 넘다 넘다 문의 친...    
859  가성 시설 크다 히터 자다 나오다 카운터 남자 직원 대처 자다 해주다 친절하다 펜트...    
857  원래 청결 예민 한편 여기다 깨끗하다 가격 저렴하다 방도 이쁘다 넘다 넘다 문의 직...    
188  친절하다 깨끗하다 건물 입구 예쁘다 깔끔하다 필요하다 갖추다 숙박 중간 청소 해주다...  
456  친절하다 깨끗하다 건물 입구 예쁘다 깔끔하다 필요하다 자다 갖추다 숙박 중간 청소 ...   
171  가성 최고봉 가격 대비 최상 퀄리팁니 업그레이드 주심 개쩜 시설 정말 친절하다 카펫...   
439  가성 최고봉 가격 대비 최상 퀄리팁니 업그레이드 주심 개쩜 위치 위치 시설 정말 직...    
